# Task 1b
#### Please note, that you are required to run the code in Task 1a first to obtain the base file for this task.

### Import required libraries

In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
import os
import shutil

### Initialise filepaths and url

In [6]:
#Read file that contains volume-issue_date-publication link data
file_read = "/Users/manasvaidya/Desktop/Python Projects/Stigler Centre RA 23/manas_vaidya/task1a/task1a_manas_vaidya.csv" 
                ##Change filepath as per your system
     
# Initialise destination of code output
file_write = 'task1b_manas_vaidya.csv' 


### Create scraping function
Note: ~3 hr runtime on my end due to high volume of scrape data.

In [7]:
# Write function to scrape data

def scrape_aer_jel(csv_file_read, csv_file_write):
    # Read csv file for publications data
    df = pd.read_csv(csv_file_read)
    
    # Initialise CSV file to be written into
    csv_file = open(csv_file_write, "w")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["volume", "issue_date", "article_title", "authors", "page_numbers", "article_link",
                        "jel_code", "jel_description"]) #Set column names



    # Extract volume, issue_date, publication url from the data frame
    for vol, iss_dt, url in zip(df.volume, df.issue_date, df.link):


        # Extract publication url contents
        source = requests.get(url).text

        # Create BeautifulSoup object of journals
        soup = BeautifulSoup(source, "lxml")

        # Filter individual articles
        for article in soup.find("section", class_ = "journal-article-group").find_all('article'):
            for sec_title in article.find_all('h3', class_='title'):
                link_article = sec_title.a.get('href')
                url_article = f'https://www.aeaweb.org{link_article}'


                # Request data from an aritcle page
                source_article = requests.get(url_article).text

                # Create BeautifulSoup object fromt he article page
                soup_article = BeautifulSoup(source_article, 'lxml')

                # Restrict data to relevant attributes section
                section_article = soup_article.find('section', class_='primary article-detail journal-article')

                #Extract title
                title = section_article.h1.text

                # Extract Auhtors
                ath = ""
                for author in section_article.find_all('li', class_='author'):  ## Modify loop to merge authors into one line
                    if author.text == "" :
                        ath = ath 
                    else:
                        ath = ath + "; " + author.text.strip()

                # Extract page numbers
                pages = section_article.find('li', class_='pages').text[1:-1]

                # Extract JEL Codes and description
                jel = section_article.find('ul', class_='jel-codes')
                if jel:
                    for j in jel.find_all('li'):
                        x = ""
                        for i in j.text.split()[1:]:
                            x=x+i+" "

                        #print(vol)
                        #print(iss_dt)
                        #print(title)
                        #print(ath[1:])
                        #print(pages)
                        #print(url_article)
                        #print(j.text.split()[0])
                        #print(x)
                        #print()
                        csv_writer.writerow([vol, iss_dt, title, ath[1:], pages, url_article, j.text.split()[0], x])
    csv_file.close()

In [8]:
# Run scraper
scrape_aer_jel(file_read, file_write)

,volume,issue_date,article_title,authors,page_numbers,article_link,jel_code,jel_description
0,Volume 112,"December 1999 (Vol. 89, No.5 )",Using Field Experiments to Test Equivalence be...,David Lucking-Reiley,pp. 1063-1080,https://www.aeaweb.org/articles?id=10.1257/aer...,D44,Auctions
1,Volume 112,"December 1999 (Vol. 89, No.5 )",The Effect of Price Advertising on Prices: Evi...,Jeffrey Milyo; Joel Waldfogel,pp. 1081-1096,https://www.aeaweb.org/articles?id=10.1257/aer...,L11,"Production, Pricing, and Market Structure; Siz..."
2,Volume 112,"December 1999 (Vol. 89, No.5 )",The Effect of Price Advertising on Prices: Evi...,Jeffrey Milyo; Joel Waldfogel,pp. 1081-1096,https://www.aeaweb.org/articles?id=10.1257/aer...,M37,Advertising
3,Volume 112,"December 1999 (Vol. 89, No.5 )",The Effect of Price Advertising on Prices: Evi...,Jeffrey Milyo; Joel Waldfogel,pp. 1081-1096,https://www.aeaweb.org/articles?id=10.1257/aer...,L66,Food; Beverages; Cosmetics; Tobacco; Wine and ...
4,Volume 112,"December 1999 (Vol. 89, No.5 )",The Effect of Price Advertising on Prices: Evi...,Jeffrey Milyo; Joel Waldfogel,pp. 1081-1096,https://www.aeaweb.org/articles?id=10.1257/aer...,L51,Economics of Regulation
...,...,...,...,...,...,...,...,...
8515,Volume 89,"March 1999 (Vol. 89, No.1 )",Social Distance and Other-Regarding Behavior i...,Iris Bohnet; Bruno S. Frey,pp. 335-339,https://www.aeaweb.org/articles?id=10.1257/aer...,C72,Noncooperative Games
8516,Volume 89,"March 1999 (Vol. 89, No.1 )",Social Distance and Other-Regarding Behavior i...,Elizabeth Hoffman; Kevin McCabe; Vernon L. Smith,pp. 340-341,https://www.aeaweb.org/articles?id=10.1257/aer...,C72,Noncooperative Games
8517,Volume 89,"March 1999 (Vol. 89, No.1 )","Optimal Inflation Targets, ""Conservative"" Cent...",Roel M. W. J. Beetsma; Henrik Jensen,pp. 342-347,https://www.aeaweb.org/articles?id=10.1257/aer...,E52,Monetary Policy
8518,Volume 89,"March 1999 (Vol. 89, No.1 )","Optimal Inflation Targets, ""Conservative"" Cent...",Roel M. W. J. Beetsma; Henrik Jensen,pp. 342-347,https://www.aeaweb.org/articles?id=10.1257/aer...,E31,Price Level; Inflation; Deflation
